# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

In [64]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='minawm_0041350787'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/Mina_Bastion_Key2' #'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Basic FABRIC Slice Configuration

In [65]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [66]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
TACC        10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
UTAH        10  318/320  2552/2560  116390/116400  635/635                 2/2                             4/4                            16/16               4/4               5/5
WASH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
MICH         6  192/192  1536/1536  60600/60600    381/38

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [67]:
# Slice 
slice_name = 'P4Lang_Tutorial'

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 40

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"

h1_ip="10.0.1.1"
h2_ip="10.0.2.2"
h3_ip="10.0.3.3"

host_cores = 2
host_ram = 8
host_disk = 10
# Sites
site_1 = 'MICH' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_2 = 'MICH' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_3 = 'MICH' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX' #'NCSA' #'STAR' #'TACC' #'STAR' #'NCSA' #'UTAH'

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'



### Create the Slice

In [68]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site_1)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_image(image)
    [s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
    [s1_iface_to_s2, s1_iface_to_s3] = s1.add_component(model='NIC_ConnectX_6', name="s1_switch_nic").get_interfaces()
    s1_iface_to_s2.set_vlan(vlan=vlan)
    s1_iface_to_s3.set_vlan(vlan=vlan)

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site_2)
    s2.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2.set_image(image)
    [s2_iface_local] = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()
    [s2_iface_to_s1, s2_iface_to_s3] = s2.add_component(model='NIC_ConnectX_5', name="s2_switch_nic").get_interfaces()
    s2_iface_to_s1.set_vlan(vlan=vlan)
    s2_iface_to_s3.set_vlan(vlan=vlan)
   
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site_3)
    s3.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3.set_image(image)
    [s3_iface_local] = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()
    [s3_iface_to_s1, s3_iface_to_s2] = s3.add_component(model='NIC_ConnectX_5', name="s3_switch_nic").get_interfaces()
    s3_iface_to_s1.set_vlan(vlan=vlan)
    s3_iface_to_s2.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    [h1_iface] = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_2)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    [h2_iface] = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()
 
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site_3)
    h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h3.set_image(image)
    [h3_iface] = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])
    
    #Submit Slice Request
    slice.submit() #(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

Waiting for slice .............. Slice state: StableOK
Waiting for ssh in slice ..... ssh successful
Running post boot config ... Done!


# timeout needs to be extended

## Get the Slice

In [69]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")  
            print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    for network in slice.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
   Cores             : 2
   RAM               : 8
   Disk              : 100
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : mich-w2.fabric-testbed.net
   Site              : MICH
   Management IP     : 2607:f018:110:11:f816:3eff:fe5e:9f0a
   Reservation ID    : b65767be-6765-4eae-9734-7f122c5e237b
   Reservation State : Active
   SSH Command       : ssh -i /home/fabric/.ssh/id_rsa -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@2607:f018:110:11:f816:3eff:fe5e:9f0a
   Components        :  
      Name             : s1-s1_switch_nic
      Details          : Mellanox ConnectX-6 VPI MCX653 dual port 100Gbps
      Disk (G)         : 0
      Units            : 1
      PCI Address      : ['0000:a1:00.0', '0000:a1:00.1']
      Model            : NIC_ConnectX_6
      Type             : SmartNIC
      Name             : s1-s1_local_nic
      Details          : Mellanox ConnectX-6 VPI MCX653 dual port 100Gbps
  

In [74]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

s1_ip_type = ""
s2_ip_type = ""
s3_ip_type = ""
h1_ip_type = ""
h2_ip_type = ""
h3_ip_type = ""

try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        if(node.get_name() == "s1"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                s1_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                s1_ip_type = "IPv6"
            else:
                s1_ip_type = "Unknown"
        if(node.get_name() == "s2"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                s2_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                s2_ip_type = "IPv6"
            else:
                s2_ip_type = "Unknown"
        if(node.get_name() == "s3"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                s3_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                s3_ip_type = "IPv6"
            else:
                s3_ip_type = "Unknown"
        if(node.get_name() == "h1"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                h1_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                h1_ip_type = "IPv6"
            else:
                h1_ip_type = "Unknown"
        if(node.get_name() == "h2"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                h2_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                h2_ip_type = "IPv6"
            else:
                h2_ip_type = "Unknown"
        if(node.get_name() == "h3"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                h3_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                h3_ip_type = "IPv6"
            else:
                h3_ip_type = "Unknown"
except Exception as e:
    print(f"Exception: {e}")
print(s1_ip_type)
print(s2_ip_type)
print(s3_ip_type)
print(h1_ip_type)
print(h2_ip_type)
print(h3_ip_type)

IPv6
IPv6
IPv6
IPv6
IPv6
IPv6


## Configure Nodes


In [91]:
host_config_script_h1 = ""
if(s1_ip_type == "IPv4"):
    host_config_scrip_h1 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git'
else:
    host_config_script_h1 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git config --global http.proxy "socks5h://localhost:4567" && git clone https://github.com/p4lang/tutorials.git'
host_config_script_h2 = ""
if(s1_ip_type == "IPv4"):
    host_config_scrip_h2 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git'
else:
    host_config_script_h2 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git config --global http.proxy "socks5h://localhost:4567" && git clone https://github.com/p4lang/tutorials.git'
host_config_script_h3 = ""
if(s1_ip_type == "IPv4"):
    host_config_scrip_h3 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git'
else:
    host_config_script_h3 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git config --global http.proxy "socks5h://localhost:4567" && git clone https://github.com/p4lang/tutorials.git'
    
try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
#     iface1.set_ip(ip=h1_ip, cidr="24")
    h1_os_iface.set_ip(ip=h1_ip, cidr="24")
    
    stdout, stderr = h1.execute(host_config_script_h1)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.set_ip(ip=h2_ip, cidr="24")
    
    
    stdout, stderr = h2.execute(host_config_script_h2)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h3 = slice.get_node(name=h3_name)
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.set_ip(ip=h3_ip, cidr="24")
    
    stdout, stderr = h3.execute(host_config_script_h3)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

stdout: 
stdout: 
stdout: 


## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [76]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
    
    file_attributes = s1.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s1.execute(f"chmod +x router_setup_p4_bmv2_container.sh sudo sh -c './router_setup_p4_bmv2_container.sh {s1_h1_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()} {s1_ip_type} > /tmp/script.log 2>&1'")
    print("stdout: {}".format(stdout))

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

stdout: ('', '')


In [80]:
try:
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)
    s2_s3_os_iface = s2.get_interface(network_name=net_s2_s3_name)
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s2.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s2_h2_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} {s2_s3_os_iface.get_os_interface()} {s2_ip_type} > /tmp/script.log 2>&1'")
    print("stdout: {}".format(stdout))
    
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

stdout: ('', '')


In [81]:
try:
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s3.get_interface(network_name=net_s2_s3_name)
     
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s3.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} {s3_s2_os_iface.get_os_interface()} {s3_ip_type} > /tmp/script.log 2>&1'")
    print("stdout: {}".format(stdout))
    
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

stdout: ('', '')


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [89]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -it fabric_p4 simple_switch_CLI  > /tmp/script.log 2>&1'")
    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 2607:f018:110:11:f816:3eff:fe5e:9f0a
s1_commands.txt
file_attributes: -rw-rw-r--   1 1000     1000          185 18 Mar 10:17 ?
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Setting default action of myTunnel_exact\naction:              drop\nruntime data:        \nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:01\naction:              myTunnel_forward\nruntime data:        00:01\nInvalid table operation (DUPLICATE_ENTRY)\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:02\naction:              myTunnel_forward\nruntime data:        00:02\nInvalid table operation (DUPLICATE_ENTRY)\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:03\naction:              myTunnel_forward\nruntime data:        00:03\nInvalid table operation (DUPLICATE_ENTRY)\nRuntimeCm

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [83]:
h1.upload_file('scripts/send.py', 'send.py')

<SFTPAttributes: [ size=1744 uid=1000 gid=1000 mode=0o100664 atime=1647598647 mtime=1647598648 ]>

In [84]:
h2.upload_file('scripts/receive.py', 'receive.py')

<SFTPAttributes: [ size=1137 uid=1000 gid=1000 mode=0o100664 atime=1647598649 mtime=1647598649 ]>

In [92]:
h1.execute('sudo mv send.py tutorials/exercises/basic_tunnel/send_modified.py')

('', '')

In [93]:
h2.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')

('', '')

## Below, we send a few packets.

In [94]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

('', '')

## And below we receive them.

In [95]:
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 06:f9:69:42:68:f1
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 2
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
        chksum    = 0x5faf
        src       = 10.20.5.17
        dst       = 10.10.2.2
        \options   \
###[ Raw ]### 
           load      = 'message100'
###[ Padding ]### 
              load      = '\x00\x00\x00\x00\x00\x00\x00\x00'

got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 06:f9:69:42:68:f1
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 2
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag    

## Delete Slice

In [96]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()